## Primera limpieza de datos

Vamos a analizar los datos generales del dataset con las columnas inferidas

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

df = map_strings.copy()

# Display basic information about the dataset
print("Dataset Shape:", df.shape)
print("\nDataset Info:")
df.info()
print("\nBasic Statistics:")
df.describe()
print("Head of the dataset:")
df.head()

No contamos con valores nulos en el dataset. Corresponden el numero de observaciones no nulas en todas las columnas con el numero total de observaciones en shape.
Usando las variables inferidas contamos con 59 variables. De las cuales muchas son binarias.

Vamos a realizar histogramas para visualizar las distribuciones de los datos.

In [ ]:
df.hist(figsize=(15, 15))
plt.show()

Podemos ver las variables binarias facilmente. Hay dos variables que vamos a eliminar: por un lado Id, que es uniforme y por otro lado Cylinders que es una constante.

Tambien podemos borrar Petrol, ya que usando Diesel y CNG ya podemos inferir que sera Petrol si ambos valores son cero.

In [ ]:
df.drop(columns=["Id", "Cylinders", "Petrol"], inplace=True)

Hay muchas features binarias con pocos valores en uno. Deberemos analizarlas bien.

Vamos a hacer la matriz de correlacion para ver que variables estan mas correlacionadas.

In [ ]:
corr_matrix = df.corr()
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Matriz de Correlación')
plt.show()

Y vamos a ver scatterplots entre las variables con una correlacion mayor a 0.5

In [ ]:
def scatter_plots(df):
    # Create scatterplots for highly correlated variables (|correlation| > 0.5)
    high_corr_pairs = []
    for i in range(df.shape[1]):
        for j in range(i+1, df.shape[1]):
            corr = corr_matrix.iloc[i,j]
            if abs(corr) > 0.5:
                high_corr_pairs.append((df.columns[i], df.columns[j], corr))

    # Sort pairs by absolute correlation value
    high_corr_pairs.sort(key=lambda x: abs(x[2]), reverse=True)

    # Create scatterplots for highly correlated pairs
    n_pairs = len(high_corr_pairs)
    n_cols = 2
    n_rows = (n_pairs + n_cols - 1) // n_cols

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
    axes = axes.flatten()

    for idx, (col1, col2, corr) in enumerate(high_corr_pairs):
        sns.scatterplot(data=df, x=col1, y=col2, ax=axes[idx])
        axes[idx].set_title(f'{col1} vs {col2}\nCorrelation: {corr:.2f}')
        axes[idx].tick_params(axis='x', rotation=45)

    # Hide empty subplots if any
    for idx in range(len(high_corr_pairs), len(axes)):
        axes[idx].set_visible(False)

    plt.tight_layout()
    plt.show()
    
scatter_plots(df)

Podemos ver como la columna m_life_months inferida a partir de la fecha de fabricacion esta perfectamente correlacionada con Age_08_04 y tiene alta correlacion con Mfg_Years.

Vamos a eliminar esta columna y a Mfg_Month que no parece tener demasiada relacion con el precio final.

In [ ]:
df.drop(columns=["m_life_months", "Mfg_Month"], inplace=True)

Radio y radio_cassette son variables que tambien cuentan con una correlacion del 0.99. No tiene sentido mantener a ambas y preferimos quedarnos con Radio por su correlacion con Precio.

In [ ]:
df.drop(columns=["Radio_cassette"], inplace=True)

Age_08_04 y Mfg_Year tambien cuentan con alta correlacion. Vamos a eliminar la primera, que tiene menos relacion con precio.

In [ ]:
df.drop(columns=["Age_08_04"], inplace=True)

Automatic y m_matic tienen una relacion alta porque representan lo mismo, eliminaremos la inferida.

El caso de m_dsl y Diesel es similar, es lo mismo pero hay casos en donde DSL no se agrego al nombre del modelo. Eliminamos la inferida.

Y para m_airco. Que solo indica que hay aire acondicionado.

In [ ]:
df.drop(columns=["m_matic", "m_dsl", "m_airco"], inplace=True)

m_vvti representa algo muy similar a valve (que une los vvti con los vvtil). Estos pueden llegar a tener influencia en el precio segun se ve. Vamos a eliminar valve para mantener la separacion de vvlit que parecen ser mas caros de lo normal.

In [ ]:
df.drop(columns=["valve"], inplace=True)

Vamos a ver los scatter plots de las variables vs el precio. Vamos a usar un alpha bajo para poder ver la superposicion de los puntos y las concentraciones de valores en las variables binarias.

In [ ]:
# Create scatterplots for each numeric variable vs. price
def scatterplots_price():
    n_cols = 3
    n_rows = (len(df.columns) + n_cols - 1) // n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
    axes = axes.flatten()

    for idx, col in enumerate(df.columns):
        sns.scatterplot(data=df, x=col, y='Price', ax=axes[idx], alpha=0.2)
        axes[idx].set_title(f'{col} vs Price')

    # Hide empty subplots if any
    for idx in range(len(df.columns), len(axes)):
        axes[idx].set_visible(False)

    plt.tight_layout()
    plt.show()

scatterplots_price()

Hay varias cuestiones interesantes. Por un lado contamos con una observacion con 16000 en cc. Vamos a ver que pasa con ella, puede ser un error de tipeo. 

In [ ]:
row = df.loc[df["cc"] == 16000]
print(row)

cc_milseis = df.loc[df["cc"] == 1600]
print(np.mean(cc_milseis["Price"]))

No parece haber nada extranio en la fila, deberia ser un error de tipeo pero su precio es muy superior a la media de precio de 1600cc. Preferimos eliminar esta observacion.

In [ ]:
df.drop(row.index, inplace=True)
df.shape

Otros interesantes son los precios por arriba de 30000

In [ ]:
price_rows = df.loc[df["Price"] > 25000]
price_rows

Como se puede ver corresponde a los modelos Verso. Estos estan muy alejados del precio de otros autos y son pocos. Por eso consideramos que son outliers que perjudican la regresion. Nuestro modelo no hara predicciones de precio para modelos Verso.

In [ ]:
df = df[df["m_mpv_verso"] != 1]
df.drop(columns=["m_mpv_verso"], inplace=True)
df.shape

Vamos a eliminar otras variables inferidas que parecen no tener significado real o son demasiado atipicos.

In [ ]:
df.drop(columns=["m_keuze_occ_uit", "m_g3", "m_b_ed", "m_sw", "m_xl", "m_pk", "m_nav", "m_ll", "m_gl", "m_comm"], inplace=True)

Un patron que queremos ver son los terra, sedan y comfort. Los comfort y sedan son siempre modelos terra. Quizas deberiamos excluir a estos de la variable terra.

In [ ]:
terra_sedan_comfort = df.loc[df["m_terra"] == 1]
terra_sedan = terra_sedan_comfort.loc[terra_sedan_comfort["m_comfort"] != 1]
terra = terra_sedan.loc[terra_sedan["m_sedan"] != 1]
print(f'Precio medio terra: {np.mean(terra["Price"])}')
print(f'Precio medio terra_sedan: {np.mean(terra_sedan["Price"])}')
print(f'Precio medio terra_sedan_comfort: {np.mean(terra_sedan_comfort["Price"])}')
terra.shape

La media cambia separando los valores, esto afecta a la variable terra. Vamos a hacer que si son sedan o comfort no sean terra.

In [ ]:
df.loc[(df["m_terra"] == 1) & ((df["m_comfort"] == 1) | (df["m_sedan"] == 1)), "m_terra"] = 0
terra = df.loc[df["m_terra"] == 1]
terra.shape

In [ ]:
# Display basic information about the dataset
print("Dataset Shape:", df.shape)
print("\nDataset Info:")
df.info()
print("\nBasic Statistics:")
df.describe()
print("Head of the dataset:")
df.head()

Vamos a aplicar algunas transformaciones para mejorar los datos.

In [ ]:
# Normalizado
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
toyota_normalized = df.copy()
columns = toyota_normalized.columns
toyota_normalized[columns] = scaler.fit_transform(toyota_normalized[columns])
toyota_normalized["Price"] = df["Price"]
toyota_normalized = pd.DataFrame(toyota_normalized, columns=columns)

En base a lo obtenido, observamos una Asimetría (skew) en las distribuciones, por lo que haremos los siguientes tratamientos:

- *Outliers*: quitamos los outliers del dataset.
- *Normalizado*: escalamos las variables con la funcion Min Max (0 a 1).
- *Transformacion*: aplicamos esta transformación a las columnas no binarias.


In [ ]:
def box_plots(data_df):
    # Create boxplots for all numeric variables
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 4*n_rows))
    axes = axes.flatten()

    for idx, col in enumerate(numeric_cols):
        sns.boxplot(data=data_df, y=col, ax=axes[idx])
        axes[idx].set_title(f'Boxplot of {col}')
        axes[idx].tick_params(axis='x', rotation=45)

    # Hide empty subplots if any
    for idx in range(len(numeric_cols), len(axes)):
        axes[idx].set_visible(False)

    plt.tight_layout()
    plt.show()
    
box_plots(df)

In [ ]:
skewed_cols = ["Price", "KM", "HP", "Weight", "Mfg_Year"]

In [ ]:
# Transformado
def show_skew_info(df, col):
    skewness = df[col].skew()
    kurtosis = df[col].kurtosis()
    print(f"Col: {col}")
    print(f"\tSkewness: {skewness}")
    print(f"\tKurtosis: {kurtosis}")
    
toyota_transformed = toyota_normalized[skewed_cols].copy()

show_skew_info(toyota_transformed, "KM")
toyota_transformed['KM'] = np.log(toyota_transformed['KM']+1)
toyota_transformed['KM'] = np.sqrt(toyota_transformed['KM'])
show_skew_info(toyota_transformed, "KM")

show_skew_info(toyota_transformed, "Weight")
toyota_transformed['Weight'] = np.log(toyota_transformed['Weight']+1)
toyota_transformed['Weight'] = np.sqrt(toyota_transformed['Weight'])
show_skew_info(toyota_transformed, "Weight")

toyota_transformed.describe().T

In [ ]:
def print_histograms_comparison_v2(original_df, toyota_normalized, transformed_df):
    numeric_cols = toyota_normalized.select_dtypes(include=[np.number]).columns
    n_cols = 3  # Original, Scaled, Transformed
    n_rows = len(numeric_cols)

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 4*n_rows))
    if n_rows == 1:
        axes = np.array([axes])  # Ensure axes is 2D

    for idx, col in enumerate(numeric_cols):
        # Histograma original
        sns.histplot(data=original_df, x=col, ax=axes[idx, 0], kde=True, color='skyblue')
        axes[idx, 0].set_title(f'Original: {col}')
        axes[idx, 0].tick_params(axis='x', rotation=45)

        # Histograma escalado
        sns.histplot(data=toyota_normalized, x=col, ax=axes[idx, 1], kde=True, color='salmon')
        axes[idx, 1].set_title(f'Escalado: {col}')
        axes[idx, 1].tick_params(axis='x', rotation=45)

        # Histograma transformado
        sns.histplot(data=transformed_df, x=col, ax=axes[idx, 2], kde=True, color='seagreen')
        axes[idx, 2].set_title(f'Transformado: {col}')
        axes[idx, 2].tick_params(axis='x', rotation=45)

    plt.tight_layout()
    plt.show()

print_histograms_comparison_v2(map_strings[skewed_cols], toyota_normalized[skewed_cols], toyota_transformed[skewed_cols])

In [ ]:
box_plots(toyota_transformed)